Este scrip es el de entrenamiento pre y post pandemia hasta el 202106, estos modelos y optimacion son en los que voy a predecir en test que es septiembre

# 1.Librerías

In [1]:
%pip install lightgbm==4.4.0
%pip install optuna==3.6.1


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\mconde\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\mconde\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

# 2.Datos

In [ ]:
#seteo carpeta de trabajo y archivo
dataset_path_1 = '/home/ingceciliaconde/buckets/b1/datasets/03'
dataset_path = '/home/ingceciliaconde/buckets/b1/datasets'
modelos_path = '/home/ingceciliaconde/buckets/b1/datasets'
db_path = '/home/ingceciliaconde/buckets/b1/datasets/db'
exp_path ='/home/ingceciliaconde/buckets/b1/exp/03'
dataset_file = 'competencia_02_fe_v1.csv.gz'


In [ ]:
#path de trabajo local para pruebas
# #seteo carpeta de trabajo y archivo
# dataset_path = 'C:/Users/mconde/Documents/DMEYF/Competencia_01'
# db_path = 'C:/Users/mconde/Documents/DMEYF/Competencia_01'
# modelos_path = 'C:/Users/mconde/Documents/DMEYF/Competencia_01'
# dataset_file = 'competencia_01.csv'


In [ ]:
print(f"{dataset_path}/{dataset_file}")



In [ ]:
# Lectura de datos.
data= pd.read_csv(f"{dataset_path}/{dataset_file}")
data.shape


In [ ]:
#semillas y valores de ganancia
semillas=[100183,200003,300017,700001,800011]
#genero 25 semillas aleatoreas para el split y agregale las semillas fijas
semillas = semillas + [np.random.randint(1000000) for i in range(25)]

ganancia_acierto = 273000
costo_estimulo = 7000

In [ ]:
print(f"Semillas: {semillas}")

In [ ]:
#meses
mes_train_all = [201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
                 201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004,
                 202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012,
                 202101, 202102, 202103, 202104, 202105, 202106]
pre_post_pandemia = [201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912, 202001, 202002, 202101, 202102, 202103, 202104, 202105, 202106]
mes_train_6 = [202101, 202102, 202103, 202104, 202105, 202106]
mes_train_12 = [202006, 202007, 202008, 202009, 202010, 202011, 202012,
                 202101, 202102, 202103, 202104, 202105, 202106]
mes_test = 202109

In [ ]:
mes_train=pre_post_pandemia #si uso 6 meses, cambiar a mes_train_6 y si uso all, cambiar a mes_train_all

## Darle valor a las clases

In [8]:
data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

En particular, sumaremos la clase BAJA+1, que es estructuralmente muy similar a BAJA+2, para aumentar los casos positivos. Luego, compararemos los resultados obtenidos con los de la clase con la que hemos estado trabajando hasta ahora.

In [9]:
data['clase_binaria1'] = 0
data['clase_binaria2'] = 0
data['clase_binaria1'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0) # solo baja +2
data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1) #ambas bajas

In [11]:
#filtrar por baja + 2 y mostrar clase ternaria clase binaria
data[data['clase_ternaria'] == 'BAJA+2'][['clase_ternaria', 'clase_binaria1', 'clase_binaria2']].head()


,clase_ternaria,clase_binaria1,clase_binaria2
1047,BAJA+2,1,1
1651,BAJA+2,1,1
1870,BAJA+2,1,1
1920,BAJA+2,1,1
2039,BAJA+2,1,1


# 3.Separa Test y Train

In [ ]:
train_data = data[data['foto_mes'].isin(mes_train)] # train_data = data[data['foto_mes'] == mes_train]
test_data = data[data['foto_mes'] == mes_test]

X_train = train_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
y_train_binaria1 = train_data['clase_binaria1'] #SOLO BAJA+2
y_train_binaria2 = train_data['clase_binaria2'] #TODAS LAS BAJAS
w_train = train_data['clase_peso']

X_test = test_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
y_test_binaria1 = test_data['clase_binaria1']
y_test_class = test_data['clase_ternaria']
w_test = test_data['clase_peso']

# 4.Funcion de Ganancia

In [13]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Parámetros del modelos.
params = {
    'objective': 'binary',
    'metric': 'gan_eval',
    'boosting_type': 'gbdt',
    'max_bin': 31,
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 0.3,
    'bagging_fraction': 0.7,
    'verbose': 0
}

# 5.Optimizacion del Modelo


In [ ]:
#optimizacion de hiperparametros
def objective(trial):

    num_leaves = trial.suggest_int('num_leaves', 8, 200), "cambie de 100 a 200"
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.3), # mas bajo, más iteraciones necesita
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 50, 4000), #cambie de 1 a 50 y cambie de 2000 a 4000
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0),
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0),

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1
    }
    train_data = lgb.Dataset(X_train,
                              label=y_train_binaria2, # eligir la clase
                              weight=w_train)
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=1000, # modificar, subit y subir... y descomentar la línea inferior
        callbacks=[lgb.early_stopping(stopping_rounds=int(50 + 5 / learning_rate[0]))],
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )
    
    # Calculo la ganancia máxima y la mejor iteración donde se obtuvo dicha ganancia.
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5



In [ ]:
#almacenamiento de los resultados
storage_name = "sqlite:///" + db_path + "optimization_lgbm_v7.db"
study_name = "exp_lgbm_v7_pre_post_pandemia_fe_v1"

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-11-17 19:04:13,338] Using an existing study with name 'exp_lgbm_v1' instead of creating a new one.


In [ ]:
study.optimize(objective, n_trials=100) # subir subir

## 5.1 Analisis de la optimizacion

In [ ]:
optuna.visualization.plot_optimization_history(study)


In [ ]:
plot_param_importances(study)

In [ ]:
plot_slice(study)

In [ ]:
plot_contour(study)

In [ ]:
plot_contour(study, params=['num_leaves','min_data_in_leaf'] )

# 7.Entrenamiento

---

Y finalmente tomamos el mejor modelo y lo entrenamos con la totalidad de los datos

In [ ]:
cantidad_meses_train = 20  # Ejemplo: 6 meses para atrás desde 06/2021
ventana = 3

In [34]:
# parametros de mejor modelo 
best_params = study.best_params
best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor model {best_iter}")
print(f"Mejores parámetros para el mejor modelo {best_params}")

Mejor cantidad de árboles para el mejor model 3
Mejores parámetros para el mejor modelo {'num_leaves': 74, 'learning_rate': 0.20998094578508172, 'min_data_in_leaf': 354, 'feature_fraction': 0.1413714678049873, 'bagging_fraction': 0.6124884045180642}


In [ ]:
from tqdm import tqdm  # Importar tqdm
# Preparamos el conjunto de datos
train_data = lgb.Dataset(X_train, label=y_train_binaria2, weight=w_train)

# Bucle para entrenar un modelo para cada semilla y guardar cada modelo
for semilla in semillas:
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': study.best_trial.params['num_leaves'],
        'learning_rate': study.best_trial.params['learning_rate'],
        'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
        'feature_fraction': study.best_trial.params['feature_fraction'],
        'bagging_fraction': study.best_trial.params['bagging_fraction'],
        'seed': semilla,  # Usar la semilla actual del bucle
        'verbose': 0
    }
  # Entrenar el modelo con los parámetros actuales
    model_lgb = lgb.train(params,train_data, num_boost_round=best_iter)

 # Construir el nombre del archivo con la semilla, cantidad de meses y ventana
    model_filename = f'lgb_v7_semilla{semilla}_{cantidad_meses_train}_{ventana}.txt'
    #dataset_out = modelos_path +'/'+ model_filename
    dataset_out= dataset_path_1 +'/'+ model_filename
    
    # Guardar el modelo en un archivo
    model_lgb.save_model(dataset_out_1)
    print(f'Modelo entrenado con semilla {semilla} y guardado en {dataset_out}')


Modelo entrenado con semilla 100183 y guardado en C:/Users/mconde/Documents/DMEYF/Competencia_01/lgb_v5_semilla100183_6_3.txt
Modelo entrenado con semilla 200003 y guardado en C:/Users/mconde/Documents/DMEYF/Competencia_01/lgb_v5_semilla200003_6_3.txt
Modelo entrenado con semilla 300017 y guardado en C:/Users/mconde/Documents/DMEYF/Competencia_01/lgb_v5_semilla300017_6_3.txt
Modelo entrenado con semilla 700001 y guardado en C:/Users/mconde/Documents/DMEYF/Competencia_01/lgb_v5_semilla700001_6_3.txt
Modelo entrenado con semilla 800011 y guardado en C:/Users/mconde/Documents/DMEYF/Competencia_01/lgb_v5_semilla800011_6_3.txt


Observamos variables mas importanes al modelo

In [ ]:
lgb.plot_importance(model_lgb, figsize=(10, 20))
plt.show()


Y si queremos tener las variables más importantes en forma de Dataframe:

In [29]:
#extraccion de feature importances
importances = model_lgb.feature_importance()
features = model_lgb.feature_name() #ian
#creo un dataframe para que sea mas facil de visualizar
importance_df = pd.DataFrame({'feature': features, 'importance': importances})
#ordeno por importancia
importance_df = importance_df.sort_values('importance', ascending=False)

#show.
importance_df.head(100)

,feature,importance
149,Visa_fechaalta,46
113,Master_Fvencimiento,32
72,mcomisiones_mantenimiento,22
142,Visa_mlimitecompra,13
52,mpayroll,11
...,...,...
31,mtarjeta_master_consumo,0
33,mprestamos_personales,0
34,cprestamos_prendarios,0
35,mprestamos_prendarios,0


# Corre el modelo

In [36]:
#8. Volvemos a leer el modelo.
#model_lgb = lgb.Booster(model_file= modelos_path + 'lgbm_{}_{}.txt'.format(cantidad_meses_train,ventana))

In [37]:
#a. Importo librería.
from kaggle.api.kaggle_api_extended import KaggleApi
#b. Configura el API de Kaggle
api = KaggleApi()
api.authenticate()

ModuleNotFoundError: No module named 'kaggle'

In [ ]:
import os
import re

# Definimos el patrón de nombre que esperamos para los archivos de modelos
pattern = re.compile(r'^lgb_v7_semilla\d+_\d+_\d+\.txt$') #modificar segun nomrbre de archivo

# Obtén la lista de archivos de modelos en el directorio que coinciden con el patrón
model_files = [f for f in os.listdir(dataset_path) if pattern.match(f)]
print(model_files)

#no lo corri fui directamente al ensamble, ya se que funciona

In [ ]:
# import time  # Asegúrate de que esta importación está en la parte superior de tu script
# # Parámetros de tu competencia y subidas
# competencia = 'dm-ey-f-2024-segunda'
# num_subida_kaggle = 1

# # Bucle sobre cada archivo de modelo
# for model_file in model_files:
#     # Carga el modelo
#     model_path = os.path.join(dataset_path, model_file)
#     model_lgb = lgb.Booster(model_file=model_path)
    
#     # Copia X_test para evitar añadir múltiples veces la columna 'Probabilidad'
#     X_test_copy = X_test.copy()

#     # Predecir con el modelo cargado
#     predicciones = model_lgb.predict(X_test_copy)
#     X_test_copy['Probabilidad'] = predicciones
#     tb_entrega = X_test_copy.sort_values(by='Probabilidad', ascending=False)

#     # Cortes candidatos y subidas
#     cortes = range(9000, 14000, 500)
#     for envios in cortes:
#         tb_entrega['Predicted'] = 0
#         tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
#         resultados = tb_entrega[['numero_de_cliente', 'Predicted']].reset_index(drop=True)
        
#         # Extraemos partes del nombre del archivo modelo para personalizar el mensaje y el nombre del archivo de subida
#         info_modelo = model_file.replace(".txt", "").split("_")  # Suponiendo nombre como 'lgb_v5_semillaX_Y_Z.txt'
#         semilla = info_modelo[2]  # Semilla usada
#         mes_train = info_modelo[3]  # Meses de entrenamiento
#         ventana = info_modelo[4]  # Ventana de tiempo
        
#         nombre_archivo = f"C2_CC_all_fev1_00{num_subida_kaggle}_semilla{semilla}.csv"
#         ruta_archivo = f"{exp_path}/{nombre_archivo}"
#         resultados.to_csv(ruta_archivo, index=False)
        
#         mensaje = f'Archivo {nombre_archivo}. Modelo: semilla{semilla}, meses train {mes_train}, DF {ventana}, fev1,  corte: {envios}'

        
#         # Envío a Kaggle
#         if num_subida_kaggle <= 15:
#             api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
#             print("Submission successful!")
#         else:
#             print("Esperamos 30 segundos...")
#             time.sleep(30)
#             api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
#             print("Submission successful!")
        
#         num_subida_kaggle += 1


# Ensamble de probabilidades

In [ ]:
import pandas as pd
import numpy as np
import os
import time

# Inicialización de DataFrame para almacenar todas las predicciones
df_predicciones = pd.DataFrame()

# Bucle sobre cada archivo de modelo para cargar y predecir
for model_file in model_files:
    model_path = os.path.join(dataset_path, model_file)
    model_lgb = lgb.Booster(model_file=model_path)
    
    # Copia X_test para evitar añadir múltiples veces la columna 'Probabilidad'
    X_test_copy = X_test.copy()
    
    # Predecir con el modelo cargado y almacenar las probabilidades
    predicciones = model_lgb.predict(X_test_copy)
    df_predicciones[model_file] = predicciones



In [ ]:
# Asegurarse de que ambos DataFrames tienen el mismo índice
if not df_predicciones.index.equals(X_test.index):
    print("Los índices no son iguales. Realineando...")
    df_predicciones.index = X_test.index


In [ ]:
# Verificar que las columnas esperadas estén presentes y que no haya NaNs
print(df_predicciones.head())
print("¿Hay valores NaN en las predicciones?:", df_predicciones.isna().any().any())

In [ ]:
# Calcular la media de las predicciones de todos los modelos
X_test['Probabilidad'] = df_predicciones.mean(axis=1)
tb_entrega = X_test.sort_values(by='Probabilidad', ascending=False)

In [ ]:
# Ver el DataFrame final para confirmar que los datos son correctos
print(X_test[['numero_de_cliente', 'Probabilidad']].head())


In [ ]:

import time
# Asegúrate de que num_subida_kaggle esté inicializado
# Parámetros de tu competencia y subidas
competencia = 'dm-ey-f-2024-tercera'
num_subida_kaggle = 1  # o cargar el último valor si estás continuando subidas previas

# Cortes candidatos y subidas
cortes = range(8000, 14000, 500)
for envios in cortes:
    tb_entrega['Predicted'] = 0
    tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
    resultados = tb_entrega[['numero_de_cliente', 'Predicted']].reset_index(drop=True)
    
    nombre_archivo = f"C3_CC_pos_prepandemia_fev1_ensamblepromedio{num_subida_kaggle}.csv"
    ruta_archivo = f"{exp_path}/{nombre_archivo}"
    resultados.to_csv(ruta_archivo, index=False)
    
    mensaje = f'Archivo {nombre_archivo}. Ensamble con modelos del 02, DF {ventana}, fev1, punto_corte: {envios}.'
    
    # Envío a Kaggle
    if num_subida_kaggle <= 15:
        api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
        print("Submission successful!")
    else:
        print("Esperamos 30 segundos...")
        time.sleep(30)
        api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
        print("Submission successful!")
    
    num_subida_kaggle += 1
